# Import requirements

In [2]:
from __future__ import division
import glob
import math
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
from scipy import stats
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Import Sanger data

In [3]:
sanger_df = pd.read_csv('/Users/greg/Desktop/TMHDSNeuroTat/all_Tat_data_df_2015_12_10.csv')
print sanger_df.shape
sanger_df.head()

(2278, 121)


,PatientID,VisitNum,DateOfVisit,Age,YS,Gender,Tissue,Protocol,iCD4,nCD4,...,Pos92,Pos93,Pos94,Pos95,Pos96,Pos97,Pos98,Pos99,Pos100,Pos101
0,A0001,R00,2006-09-12,51,10.767123,Male,NaN,NaN,611,301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0001,R01,2007-08-15,52,11.690411,Male,PBMC,Genomic,611,301,...,-,-,-,-,-,-,-,-,-,-
2,A0001,R02,2008-06-04,53,12.495890,Male,PBMC,Genomic,611,301,...,-,-,-,-,-,-,-,-,-,-
3,A0001,R03,2008-11-11,53,12.934247,Male,PBMC,Genomic,611,301,...,E,R,E,T,E,T,D,P,V,D
4,A0001,R04,2009-11-10,54,13.931507,Male,PBMC,Genomic,611,301,...,E,R,E,T,E,T,D,P,F,D


# Filter dataframe rubbish out

In [4]:
# include only genomic sequences
m2 = sanger_df['Protocol'] == 'Genomic'
sanger_df = sanger_df[m2]
print sanger_df.shape

# include only PBMC sequences
m1 = sanger_df['Tissue'] == 'PBMC'
sanger_df = sanger_df[m1]
print sanger_df.shape

# include only sequences with a TMHD score
m3 = ~sanger_df['TMHDS'].isnull()
sanger_df = sanger_df[m3]
print sanger_df.shape

m4 = ~sanger_df['GDS'].isnull()
gds_df = sanger_df[m4]
print gds_df.shape

(918, 121)
(754, 121)
(591, 121)
(70, 121)


In [5]:
for c in sanger_df.columns:
    print c,

PatientID VisitNum DateOfVisit Age YS Gender Tissue Protocol iCD4 nCD4 CD4 iCD8 nCD8 CD8 iVL pVL VL ART TMHDS GDS Pos01 Pos02 Pos03 Pos04 Pos05 Pos06 Pos07 Pos08 Pos09 Pos10 Pos11 Pos12 Pos13 Pos14 Pos15 Pos16 Pos17 Pos18 Pos19 Pos20 Pos21 Pos22 Pos23 Pos24 Pos25 Pos26 Pos27 Pos28 Pos29 Pos30 Pos31 Pos32 Pos33 Pos34 Pos35 Pos36 Pos37 Pos38 Pos39 Pos40 Pos41 Pos42 Pos43 Pos44 Pos45 Pos46 Pos47 Pos48 Pos49 Pos50 Pos51 Pos52 Pos53 Pos54 Pos55 Pos56 Pos57 Pos58 Pos59 Pos60 Pos61 Pos62 Pos63 Pos64 Pos65 Pos66 Pos67 Pos68 Pos69 Pos70 Pos71 Pos72 Pos73 Pos74 Pos75 Pos76 Pos77 Pos78 Pos79 Pos80 Pos81 Pos82 Pos83 Pos84 Pos85 Pos86 Pos87 Pos88 Pos89 Pos90 Pos91 Pos92 Pos93 Pos94 Pos95 Pos96 Pos97 Pos98 Pos99 Pos100 Pos101


In [6]:
c = ['PatientID', 'VisitNum', 'DateOfVisit', 'Age', 'YS',
 'Gender', 'iCD4', 'nCD4', 'CD4', 'iCD8', 'nCD8', 'CD8',
 'iVL', 'pVL', 'VL', 'ART', 'TMHDS']

demo_df = sanger_df[c]
demo_df['pVL'] = np.log10(demo_df['pVL'])
demo_df['iVL'] = np.log10(demo_df['iVL'])
demo_df['VL'] = np.log10(demo_df['VL'])
demo_df.describe()

/Users/greg/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/greg/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/greg/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/index

,Age,YS,iCD4,nCD4,CD4,iCD8,nCD8,CD8,iVL,pVL,VL,TMHDS
count,591.000000,590.000000,589.000000,589.000000,589.000000,582.000000,582.000000,582.000000,588.000000,588.000000,588.000000,591.000000
mean,48.436548,15.254897,396.095076,236.504244,556.142615,916.482818,602.761168,1222.128866,3.804486,4.646559,2.285481,9.549069
std,8.220208,7.729558,292.807344,189.096029,313.798416,427.569896,315.419546,768.928576,1.296233,1.038886,1.177502,2.519442
min,22.000000,0.038356,0.000000,0.000000,6.000000,51.000000,51.000000,89.000000,1.681241,1.681241,1.301030,0.500000
25%,43.000000,9.486986,195.000000,88.000000,351.000000,640.000000,385.000000,737.250000,2.617000,4.260137,1.681241,8.000000
50%,48.000000,14.906849,348.000000,220.000000,532.000000,821.000000,557.000000,1040.000000,4.007585,4.898034,1.681241,10.000000
75%,53.500000,20.977397,570.000000,353.000000,740.000000,1137.000000,781.000000,1537.000000,4.966184,5.250142,2.748188,12.000000
max,72.000000,40.536986,1951.000000,957.000000,3269.000000,2972.000000,2611.000000,7286.000000,6.421604,7.380211,6.322067,12.000000


In [7]:
Counter(demo_df['ART'])

Counter({nan: 1, 'naive': 40, 'non-adherent': 5, 'off': 36, 'on': 509})

In [8]:
demo_df.shape

(591, 17)

# Diversity function

In [9]:
# calculate diversity using a dictionary object as input
def diversity_column(counts, hill):
    vals = np.array(counts.values(), dtype=np.float64)
    vals /= vals.sum()
    if hill == 1:
        return np.exp(-np.sum(vals*np.log(vals)))
    else:
        return (vals**hill).sum()**(1/(1-hill))

# Statistical test functions

In [10]:
def perform_Welch_Ttest(variant_frame1, variant_frame2, attribute):
    a1 = np.array(variant_frame1[attribute])
    a2 = np.array(variant_frame2[attribute])
    a1 = a1[~np.isnan(a1)]
    a2 = a2[~np.isnan(a2)]
    num_a1 = sum(np.isfinite(a1))
    num_a2 = sum(np.isfinite(a2))
    mean1 = np.mean(a1)
    mean2 = np.mean(a2)
    t_stat, p_val = stats.ttest_ind(a1, a2, equal_var = False)
    return t_stat, p_val, (num_a1, num_a2), (mean1, mean2)

def perform_ks_test(variant_frame1, variant_frame2, attribute):
    a1 = np.array(variant_frame1[attribute])
    a2 = np.array(variant_frame2[attribute])
    a1 = a1[~np.isnan(a1)]
    a2 = a2[~np.isnan(a2)]
    num_a1 = sum(np.isfinite(a1))
    num_a2 = sum(np.isfinite(a2))
    ks_stat, p_val = stats.ks_2samp(a1, a2)
    return ks_stat, p_val, (num_a1, num_a2)

def perform_mannwhitneyU_test(variant_frame1, variant_frame2, attribute):
    a1 = np.array(variant_frame1[attribute])
    a2 = np.array(variant_frame2[attribute])
    a1 = a1[~np.isnan(a1)]
    a2 = a2[~np.isnan(a2)]
    num_a1 = sum(np.isfinite(a1))
    num_a2 = sum(np.isfinite(a2))
    med_a1 = np.median(a1)
    med_a2 = np.median(a2)
    u_stat, p_val = stats.mannwhitneyu(a1, a2)
    return u_stat, p_val, (med_a1, med_a2), (num_a1, num_a2)

#chi square test code
def perform_chi2_test(variant_frame1, variant_frame2, attribute, threshold):
    a1 = np.array(variant_frame1[attribute])
    a2 = np.array(variant_frame2[attribute])
    a1 = a1[~np.isnan(a1)]
    a2 = a2[~np.isnan(a2)]
    num_a1 = sum(np.isfinite(a1))
    num_a2 = sum(np.isfinite(a2))
    group1count1, group1count2 = sum(a1 < threshold), sum(a1 >= threshold)
    group2count1, group2count2 = sum(a2 < threshold), sum(a2 >= threshold)
    obs = np.array([[group1count1, group1count2], [group2count1, group2count2]])
    chi2, p_val, dof, expect = stats.chi2_contingency(obs, correction=False)
    return p_val, (num_a1, num_a2), (chi2, p_val, dof, expect)

# Execution of code

# TMHDS Stats

In [11]:
aa_list = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
clin_cols = ['PatientID', 'VisitNum', 'DateOfVisit', 'Age', 'YS', 'Gender',
             'Tissue', 'Protocol', 'iCD4', 'nCD4', 'CD4','iCD8', 'nCD8', 'CD8',
             'iVL', 'pVL', 'VL', 'ART', 'TMHDS', 'GDS']

position_dict = {'Position':[], 'Richness':[], 'Diversity':[], 'Variants':[]}

total_tests_performed = 0

stats_dict = {'Position':[], 'Variant':[], 'Frequency':[],
              'TMHDS Mean':[], 'TMHDS Median': [], 'TMHDS Std':[],
              'Mann Whitney U P-value':[],
              'KS Test P-value':[],
              "Welch's T-Test P-value":[]}

for i in np.arange(1,102):
    position_col = 'Pos%02i' % i
    p = sanger_df[position_col]
    position_mask = p.isin(aa_list)
    position_variants = p[position_mask]
    variants_dict = Counter(position_variants)
    richness = diversity_column(variants_dict, 0)
    diversity = diversity_column(variants_dict, 1)
    cols = clin_cols + [position_col]
    position_frame = sanger_df[position_mask][cols]
    # append to position_dict
    position_dict['Position'].append(i)
    position_dict['Richness'].append(richness)
    position_dict['Diversity'].append(diversity)
    position_dict['Variants'].append(variants_dict)
    
    L1 = variants_dict.items()
    for combo in combinations(L1, 2):
        variant1, variant2 = combo[0][0], combo[1][0]
        count1, count2 = combo[0][1], combo[1][1]
        if min([count1, count2]) > 20:
            
            variant_mask1 = position_frame[position_col] == variant1[0]
            variant_mask2 = position_frame[position_col] == variant2[0]
            variant_frame1 = position_frame[variant_mask1]
            variant_frame2 = position_frame[variant_mask2]
            t1 = np.array(variant_frame1['TMHDS'])
            t2 = np.array(variant_frame2['TMHDS'])
            
            total_tests_performed += 1
            u_stat, mann_whitney_p_val = stats.mannwhitneyu(t1, t2)
            ks_stat, ks_p_val = stats.ks_2samp(t1, t2)
            t_stat, welch_p_val = stats.ttest_ind(t1, t2, equal_var = False)
            
            group1count1, group1count2 = sum(t1 < 10), sum(t1 >= 10)
            group2count1, group2count2 = sum(t2 < 10), sum(t2 >= 10)
            obs = np.array([[group1count1, group1count2], [group2count1, group2count2]])
            chi2, p_val, dof, expect = stats.chi2_contingency(obs, correction=False)
            #print position_col, p_val,
             
            # first variant
            stats_dict['Position'].append(position_col)
            stats_dict['Variant'].append(combo[0][0])
            stats_dict['Frequency'].append(combo[0][1])
            stats_dict['TMHDS Mean'].append(np.mean(t1))
            stats_dict['TMHDS Median'].append(np.median(t1))
            stats_dict['TMHDS Std'].append(np.std(t1))
            stats_dict['Mann Whitney U P-value'].append(mann_whitney_p_val)
            stats_dict['KS Test P-value'].append(ks_p_val)
            stats_dict["Welch's T-Test P-value"].append(welch_p_val)
            # second variant
            stats_dict['Position'].append(position_col)
            stats_dict['Variant'].append(combo[1][0])
            stats_dict['Frequency'].append(combo[1][1])
            stats_dict['TMHDS Mean'].append(np.mean(t2))
            stats_dict['TMHDS Median'].append(np.median(t2))
            stats_dict['TMHDS Std'].append(np.std(t2))
            stats_dict['Mann Whitney U P-value'].append(mann_whitney_p_val)
            stats_dict['KS Test P-value'].append(ks_p_val)
            stats_dict["Welch's T-Test P-value"].append(welch_p_val)
            
print total_tests_performed
col_order = ['Position','Variant','Frequency','TMHDS Mean', 'TMHDS Median',
             'TMHDS Std', 'Mann Whitney U P-value', 'KS Test P-value', "Welch's T-Test P-value"]
TMHDS_statsdf = pd.DataFrame(stats_dict)[col_order]
TMHDS_statsdf

50


,Position,Variant,Frequency,TMHDS Mean,TMHDS Median,TMHDS Std,Mann Whitney U P-value,KS Test P-value,Welch's T-Test P-value
0,Pos03,P,414,9.466184,10.00,2.578491,0.036239,0.352879,0.168697
1,Pos03,S,50,8.940000,9.00,2.491265,0.036239,0.352879,0.168697
2,Pos06,P,455,9.416484,10.00,2.544905,0.248806,0.957019,0.811254
3,Pos06,T,24,9.562500,10.25,2.844265,0.248806,0.957019,0.811254
4,Pos08,Q,29,9.396552,10.00,2.630653,0.475185,0.999075,0.882027
5,Pos08,L,445,9.473034,10.00,2.507148,0.475185,0.999075,0.882027
6,Pos10,P,426,9.415493,10.00,2.609552,0.329282,0.433067,0.804414
7,Pos10,T,44,9.500000,9.75,2.067058,0.329282,0.433067,0.804414
8,Pos19,K,301,9.441860,10.00,2.672757,0.176719,0.506767,0.762143
9,Pos19,Q,164,9.368902,10.00,2.361313,0.176719,0.506767,0.762143


# GDS stats

In [116]:
aa_list = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
clin_cols = ['PatientID', 'VisitNum', 'DateOfVisit', 'Age', 'YS', 'Gender',
             'Tissue', 'Protocol', 'iCD4', 'nCD4', 'CD4','iCD8', 'nCD8', 'CD8',
             'iVL', 'pVL', 'VL', 'ART', 'TMHDS', 'GDS']

position_dict = {'Position':[], 'Richness':[], 'Diversity':[], 'Variants':[]}

total_tests_performed = 0

stats_dict = {'Position':[], 'Variant':[], 'Frequency':[],
              'GDS Mean':[], 'GDS Median': [], 'GDS Std':[],
              'Mann Whitney U P-value':[],
              'KS Test P-value':[],
              "Welch's T-Test P-value":[]}

for i in np.arange(1,102):
    position_col = 'Pos%02i' % i
    p = sanger_df[position_col]
    position_mask = p.isin(aa_list)
    position_variants = p[position_mask]
    variants_dict = Counter(position_variants)
    richness = diversity_column(variants_dict, 0)
    diversity = diversity_column(variants_dict, 1)
    cols = clin_cols + [position_col]
    position_frame = sanger_df[position_mask][cols]
    # append to position_dict
    position_dict['Position'].append(i)
    position_dict['Richness'].append(richness)
    position_dict['Diversity'].append(diversity)
    position_dict['Variants'].append(variants_dict)
    
    L1 = variants_dict.items()
    for combo in combinations(L1, 2):
        variant1, variant2 = combo[0][0], combo[1][0]
        count1, count2 = combo[0][1], combo[1][1]
        if min([count1, count2]) > 20:
            
            variant_mask1 = position_frame[position_col] == variant1[0]
            variant_mask2 = position_frame[position_col] == variant2[0]
            variant_frame1 = position_frame[variant_mask1]
            variant_frame2 = position_frame[variant_mask2]
            t1 = np.array(variant_frame1['GDS'])
            t2 = np.array(variant_frame2['GDS'])
            t1 = t1[~np.isnan(t1)]
            t2 = t2[~np.isnan(t2)]
            
            if min([len(t1), len(t2)]) > 10:
            
                total_tests_performed += 1
                u_stat, mann_whitney_p_val = stats.mannwhitneyu(t1, t2)
                ks_stat, ks_p_val = stats.ks_2samp(t1, t2)
                t_stat, welch_p_val = stats.ttest_ind(t1, t2, equal_var = False)

                #group1count1, group1count2 = sum(t1 < 10), sum(t1 >= 10)
                #group2count1, group2count2 = sum(t2 < 10), sum(t2 >= 10)
                #obs = np.array([[group1count1, group1count2], [group2count1, group2count2]])
                #chi2, p_val, dof, expect = stats.chi2_contingency(obs, correction=False)
                #print position_col, p_val,

                # first variant
                stats_dict['Position'].append(position_col)
                stats_dict['Variant'].append(combo[0][0])
                stats_dict['Frequency'].append(len(t1))
                stats_dict['GDS Mean'].append(np.mean(t1))
                stats_dict['GDS Median'].append(np.median(t1))
                stats_dict['GDS Std'].append(np.std(t1))
                stats_dict['Mann Whitney U P-value'].append(mann_whitney_p_val)
                stats_dict['KS Test P-value'].append(ks_p_val)
                stats_dict["Welch's T-Test P-value"].append(welch_p_val)
                # second variant
                stats_dict['Position'].append(position_col)
                stats_dict['Variant'].append(combo[1][0])
                stats_dict['Frequency'].append(len(t2))
                stats_dict['GDS Mean'].append(np.mean(t2))
                stats_dict['GDS Median'].append(np.median(t2))
                stats_dict['GDS Std'].append(np.std(t2))
                stats_dict['Mann Whitney U P-value'].append(mann_whitney_p_val)
                stats_dict['KS Test P-value'].append(ks_p_val)
                stats_dict["Welch's T-Test P-value"].append(welch_p_val)
            
print total_tests_performed
col_order = ['Position','Variant','Frequency','GDS Mean', 'GDS Median',
             'GDS Std', 'Mann Whitney U P-value', 'KS Test P-value', "Welch's T-Test P-value"]
GDS_statsdf = pd.DataFrame(stats_dict)[col_order]
GDS_statsdf

9


,Position,Variant,Frequency,GDS Mean,GDS Median,GDS Std,Mann Whitney U P-value,KS Test P-value,Welch's T-Test P-value
0,Pos03,P,48,0.886963,0.583333,0.855470,0.132475,0.650641,0.439430
1,Pos03,S,15,1.074074,0.916667,0.759282,0.132475,0.650641,0.439430
2,Pos20,A,51,0.942583,0.666667,0.891462,0.415774,0.781502,0.738663
3,Pos20,T,17,0.879234,1.000000,0.561427,0.415774,0.781502,0.738663
4,Pos24,N,37,0.824529,0.583333,0.644955,0.251639,0.845229,0.346729
5,Pos24,T,16,1.095486,0.875000,1.006727,0.251639,0.845229,0.346729
6,Pos31,C,57,0.938437,0.666667,0.865365,0.401241,0.842206,0.729542
7,Pos31,R,11,0.866162,0.833333,0.539908,0.401241,0.842206,0.729542
8,Pos38,F,53,0.991395,0.750000,0.846735,0.181147,0.736166,0.282850
9,Pos38,L,14,0.748196,0.541667,0.676857,0.181147,0.736166,0.282850


# Format and save

In [101]:
m = TMHDS_statsdf['Mann Whitney U P-value'] < 0.05
DF = TMHDS_statsdf[m]
DF.to_csv('/Users/greg/Desktop/TatFigureStuff/mannwhitneytable.csv')

# Next Section

In [374]:
attribute = 'TMHDS'
position_col = 'Pos61'
threshold = 10

#####################################

aa_list = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
clin_cols = ['PatientID', 'VisitNum', 'DateOfVisit', 'Age', 'YS', 'Gender',
             'Tissue', 'Protocol', 'iCD4', 'nCD4', 'CD4','iCD8', 'nCD8', 'CD8',
             'iVL', 'pVL', 'VL', 'ART', 'TMHDS', 'GDS']

#####################################

p = sanger_df[position_col]
position_mask = p.isin(aa_list)
position_variants = p[position_mask]
variants_dict = Counter(position_variants)
richness = diversity_column(variants_dict, 0)
diversity = diversity_column(variants_dict, 1)
cols = clin_cols + [position_col]
position_frame = sanger_df[position_mask][cols]
print 'Dataframe shape:', position_frame.shape

#####################################

print variants_dict, '\n'

#####################################

L1 = variants_dict.items()
for combo in combinations(L1, 2):
    variant1, variant2 = combo[0], combo[1]
    
    variant_mask1 = position_frame[position_col] == variant1[0]
    variant_mask2 = position_frame[position_col] == variant2[0]
    variant_frame1 = position_frame[variant_mask1]
    variant_frame2 = position_frame[variant_mask2]
    if variant_frame1.shape[0] > 10 and variant_frame2.shape[0] > 10:
        print variant1, variant2
        
        #perform_chi2_test(variant_frame1, variant_frame2, 'TMHDS', 10)
        
        a1 = np.array(variant_frame1[attribute])
        a2 = np.array(variant_frame2[attribute])
        a1 = a1[~np.isnan(a1)]
        a2 = a2[~np.isnan(a2)]
        group1count1, group1count2 = sum(a1 < threshold), sum(a1 >= threshold)
        group2count1, group2count2 = sum(a2 < threshold), sum(a2 >= threshold)
        obs = np.array([[group1count1, group1count2], [group2count1, group2count2]])
        chi2, p_val, dof, expect = stats.chi2_contingency(obs, correction=False)
        print p_val, dof
        print expect
        print obs
        print ''


Dataframe shape: (481, 21)
Counter({'S': 394, 'R': 51, 'N': 19, 'C': 6, 'K': 6, 'H': 3, 'I': 1, 'Q': 1}) 

('N', 19) ('S', 394)
0.859969949933 1
[[   8.37288136   10.62711864]
 [ 173.62711864  220.37288136]]
[[  8  11]
 [174 220]]

('N', 19) ('R', 51)
0.938155510901 1
[[  8.14285714  10.85714286]
 [ 21.85714286  29.14285714]]
[[ 8 11]
 [22 29]]

('S', 394) ('R', 51)
0.889634241956 1
[[ 173.53707865  220.46292135]
 [  22.46292135   28.53707865]]
[[174 220]
 [ 22  29]]

